In [6]:
from typing import TypedDict, Annotated

from dotenv import load_dotenv

load_dotenv()

from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages

from chains import generate_chain, reflect_chain


class MessageGraph(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


REFLECT = "reflect"
GENERATE = "generate"

def fancy_box(text):
    print(text)



def generation_node(state: MessageGraph):
    return {"messages": [generate_chain.invoke({"messages": state["messages"]})]}


def reflection_node(state: MessageGraph):
    last_message = state["messages"][-1]
    fancy_box(last_message.content)
    res = reflect_chain.invoke({"messages": state["messages"]})
    fancy_box(res.content)
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(state_schema=MessageGraph)
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)


def should_continue(state: MessageGraph):
    if len(state["messages"]) > 6:
        return END
    return REFLECT


builder.add_conditional_edges(GENERATE, should_continue,{END: END, REFLECT: REFLECT})
builder.add_edge(REFLECT, GENERATE)

graph = builder.compile()
print(graph.get_graph().draw_mermaid())

graph.get_graph().draw_mermaid_png()


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	reflect(reflect)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	generate -.-> __end__;
	generate -.-> reflect;
	reflect --> generate;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xe1\x00\x00\x00\xf9\x08\x02\x00\x00\x00^T\xc5\xdb\x00\x00\x10\x00IDATx\x9c\xec\x9d\x07@\x13\xe7\xfb\xc7\xdf\xbbK\xc2\xdeSD\x96HE\xc4\xd1\xa2\xd5\xba\xf7j]\xb5\xee\xbag\xd5:\xabu\xf4\xdf\xbaj\x9d\xf5\xa7\xd6\x81\xd6-\xc5Y\xb5\x0e\xdc\xb6n\xdc\xb8\x17(C\x01\xd9\x84\x95\x90\xdc\xffI\x0e0@\x12\x83\x92\xe4.\xf7~\x8a\xe9\xf1\x8eK\xb8|\xefy\xdf\xf7y\xdf\xf79\x01M\xd3\x08\x83a1\x02\x84\xc1\xb0\x1b\xacQ\x0c\xdb\xc1\x1a\xc5\xb0\x1d\xacQ\x0c\xdb\xc1\x1a\xc5\xb0\x1d\xacQ\x0c\xdb\xc1\x1a\xd5\xc8\xads\x99o\xa2\xf3r2\x0b\x0b\xa5ri~i\x0f\x1d\x81\x10\x8dhB\xf1\x9fj"\xa1\xfc\x8d\x96\x17\'P\x88\x96)S \x8b.U\x97 \x99b\xf0\x8f,I,\x03S\x1dJ\xca\xe5\x8a\x13\xbcKg\xea\x924\x92\x13\xaa\xe5I\x11!\xa0H\x81\x089\xba\x8b\x82B\xec\xdd\xfdE\xc8$ \xb0\x7f\xb4\x0c\x11\xdb\x92\xe3\x9e\xe6H\nd\x94\x80\x14\x99\x93"\x11I\t\x90\xb4@^\xaa\x10\x88\x11\xae\x1b\xf3\xfa.\x11\x12\x14ZS\xd1(A\xcbheq\x95\xeb\\\xa4Q\x82\x96\xd3j\xa5Y"}\xa6\xbaB\x91t\xa97*\xaaK!$+UQhF\xc9\xe5(/\xa7\x

In [7]:
inputs = {
        "messages": [
            HumanMessage(
                content="""
Generate a engaging tweet about the importance of AI, but the wave if AI is making peole implement non AI changes as AI.
There should be a real usecase for AI and not for fancy AI work.
Most works we see are either acheived via work flow or simple llm and not an agentic approach.
I see many examples of analysis past history for few aggregators using agents which is like killing an ant with a sledge hammer.

                                  """
            )
        ]
    }
response = graph.invoke(inputs)
print(response)

d:\Kiran\AI Agents\langgraph\agent\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


"AI should be a tool, not a spectacle. Let's focus on real use cases where AI adds genuine value, not just for show. It's concerning when businesses implement non-AI changes in the name of AI. Let's remember the goal is efficiency, not unnecessary complexity. #AI #Tech"
Your tweet effectively conveys the importance of using AI for practical purposes rather than just for show. The message is clear and concise, making it easily understandable for your audience. However, here are some recommendations to enhance your tweet:

1. **Simplify the Language**: While the message is clear, simplifying the language a bit can help increase accessibility and engagement. Not everyone may be familiar with terms like "agentic approach" or "analysis past history for few aggregators."

2. **Add a Call to Action**: Encourage your followers to share their thoughts or experiences related to AI implementation. This can help increase engagement and make your tweet more interactive.

3. **Include Visuals**: Con

In [15]:
from pprint import pprint
pprint(response['messages'])

[HumanMessage(content='\nGenerate a engaging tweet about the importance of AI, but the wave if AI is making peole implement non AI changes as AI.\nThere should be a real usecase for AI and not for fancy AI work.\nMost works we see are either acheived via work flow or simple llm and not an agentic approach.\nI see many examples of analysis past history for few aggregators using agents which is like killing an ant with a sledge hammer.\n\n                                  ', additional_kwargs={}, response_metadata={}, id='0973cf12-c23f-407a-812b-caf358942e6a'),
 AIMessage(content='"AI should be a tool, not a spectacle. Let\'s focus on real use cases where AI adds genuine value, not just for show. It\'s concerning when businesses implement non-AI changes in the name of AI. Let\'s remember the goal is efficiency, not unnecessary complexity. #AI #Tech"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 147, 'total_tokens': 209